# 测试百度API

In [1]:
import requests
import random
import json
from hashlib import md5

import json
import os
# 导入进度条库
from tqdm import tqdm
from langdetect import detect

In [8]:
# Set your own appid/appkey.
appid = '20221016001395046'
appkey = 'W2hCHXMvlnJhdcIrlM6W'

# For list of language codes, please refer to `https://api.fanyi.baidu.com/doc/21`
from_lang = 'auto'
to_lang =  'en'

endpoint = 'http://api.fanyi.baidu.com'
path = '/api/trans/vip/translate'
url = endpoint + path
query = '东电：福岛第一核电站1号机组反应堆容器可能穿孔东电：福岛第一核电站1号机组反应堆容器可能穿孔'
# Generate salt and sign
def make_md5(s, encoding='utf-8'):
    return md5(s.encode(encoding)).hexdigest()

def baidu_translate(query,from_lang,to_lang):
    is_scusseful = False
    salt = random.randint(32768, 65536)
    sign = make_md5(appid + query + str(salt) + appkey)
    # Build request
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}
    payload = {'appid': appid, 'q': query, 'from': from_lang, 'to': to_lang, 'salt': salt, 'sign': sign}
    # Send request
    r = requests.post(url, params=payload, headers=headers)
    result = r.json()
    if "trans_result" not in result:
        return query,is_scusseful
    else:
        is_scusseful = True
        return result["trans_result"][0]['dst'],is_scusseful
print(baidu_translate(query,from_lang, to_lang)[0])

TEPCO: Possible perforation of reactor vessel at Unit 1 of Fukushima Daiichi Nuclear Power Plant


# 探索微博数据

In [2]:
def get_directories(path):
    return [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
current_directory = os.getcwd()
directories = get_directories(current_directory)
print(directories)

['.idea', '.ipynb_checkpoints', 'data', 'src']


In [26]:
WEIBO_POST_PATH = "./data/WeiboData/post"
WEIBO_ACCOUNT_PATH = "./data/WeiboData/account"
post_all_data = []
for filename in tqdm(os.listdir(WEIBO_POST_PATH)):
    if filename.endswith('.json'):
        file_path = os.path.join(WEIBO_POST_PATH, filename)
        with open(file_path, 'r',encoding='utf-8') as file:
            data_group = json.load(file)
            post_all_data.extend(data_group)
print(len(post_all_data))
# 查找其中空的数据，并删除
post_all_data = [data for data in post_all_data if "content" in data]
print(len(post_all_data))

100%|██████████| 2472/2472 [00:01<00:00, 1931.99it/s]

234391
200739


## 统计语言数量，选择其中所有中文

In [29]:
lang_dict = {}
for data in tqdm(post_all_data):
    lang = data['cont_languages']
    if lang in lang_dict:
        lang_dict[lang] += 1
    else:
        lang_dict[lang] = 1
# 去除其中非zh-cn的数据
post_all_data = [data for data in post_all_data if data['cont_languages'] == 'zh-cn']
print(len(post_all_data))

100%|██████████| 200739/200739 [00:00<00:00, 1857258.74it/s]

200676


In [28]:
lang_dict

{'zh-cn': 200676, 'ru': 2, 'zh-tw': 5, 'ja': 50, 'es': 1, 'no': 4, 'ko': 1}

In [30]:

# Set your own appid/appkey.
appid = '20221016001395046'
appkey = 'W2hCHXMvlnJhdcIrlM6W'
endpoint = 'http://api.fanyi.baidu.com'
path = '/api/trans/vip/translate'
url = endpoint + path

output_file = "translated_posts.txt"
idx = 0
# 注意使用追加模式打开文件
with open(output_file, "a", encoding="utf-8") as file:
    for post in tqdm(post_all_data):
        try:
            translated_content, success = baidu_translate(post['content'], from_lang='zh', to_lang='en')
            if success:
                post['content_trans'] = translated_content
                file.write(str(post) + "\n")
                if idx > 10:
                    break
            else:
                print(f"Translation failed for post: {post}")
                raise Exception("Translation failed")
        except Exception as e:
            print(f"Translation failed for post: {post}, Error: {e}")

  0%|          | 418/200676 [09:15<553:25:31,  9.95s/it]

Translation failed for post: {'cnt_retweet': '31', 'cont_languages': 'zh-cn', 'publish_time': '20230414094011', 'page_action_type': '07', 'cnt_agree': '5', 'nickname': '小央视频', 'topic_id': '4890361115445131', 'userid': '6362632222', 'cnt_comment': '29', 'content': '【#日专家称核污水放射性物质浓度极高# 部分超标百倍！】#福岛核污水部分放射性物质超标百倍#日本福岛大学教授柴崎直明是福岛废炉安全监视协议会委员，负责福岛第一核电站周边地质与地下水监测工作。他在接受总台记者采访时表示，目前福岛核污染水中含有的放射性物质浓度极高，未达到向海洋排放标准。 http://t.cn/A6N5Aqus', 'url': 'https://m.weibo.cn/status/4890361115445131', 'username': '小央视频'}, Error: Expecting value: line 1 column 1 (char 0)


  0%|          | 747/200676 [16:59<560:01:30, 10.08s/it]

Translation failed for post: {'cnt_retweet': '0', 'cont_languages': 'zh-cn', 'page_action_type': '11', 'relevant_username': '观察者网', 'userid': '7785349790', 'content': '#日专家称核污水放射性物质浓度极高# RT:@观察者网 【#日专家称核污水放射性物质浓度极高# 部分超标百倍！】#福岛核污水部分放射性物质超标百倍#日本福岛大学教授柴崎直明是福岛废炉安全监视协议会委员，负责福岛第一核电站周边地质与地下水监测工作。他在接受总台记者采访时表示，目前福岛核污染水中含有的放射性物质浓度极高，未达到向海洋排放标准。 \u200b\u200b\u200b ...展开', 'url': 'https://m.weibo.cn/status/4891111308923037', 'relevant_nickname': '观察者网', 'relevant_userid': '1887344341', 'publish_time': '20230416112111', 'cnt_agree': '0', 'nickname': '五千镜缘', 'cnt_comment': '0', 'username': '五千镜缘'}, Error: Expecting value: line 1 column 1 (char 0)


  0%|          | 950/200676 [21:22<75:18:14,  1.36s/it] 

...

Translation failed for post: {'cnt_retweet': '0', 'cont_languages': 'zh-cn', 'page_action_type': '07', 'userid': '2292896411', 'content': '【#福岛核污染水溅射总量预计达数升#】据日本广播协会（NHK）10月30日报道，日本东京电力公司当天称，福岛第一核电站本月25日发生的放射性废液溅射污染事件中，喷溅出的废液总量据新的推算在数升左右，而此前的推算为100毫升左右。新的推算值较此前高出数十倍。25日，在处理福岛第一核电站核污染水放射性物质的过程中，由于水管脱落导致放射性废液溅出，5名工作人员接触到了放射性废液。其中2人因身体表面的辐射量一直没有降低到安全标准值以下，一度被送往医院进行处理。（央视新闻客户端）', 'url': 'https://m.weibo.cn/status/4962665976039069', 'page_picture_path': 'https://wx4.sinaimg.cn/wap180/88aad29bly1hjdnfbr5wwj20g80d4dio.jpg', 'publish_time': '20231030221353', 'cnt_agree': '2', 'nickname': '中工网', 'topic_id': '4962665976039069', 'cnt_comment': '0', 'username': '中工网'}, Error: Expecting value: line 1 column 1 (char 0)


  1%|▏         | 2555/200676 [1:12:32<71:42:44,  1.30s/it] 

Translation failed for post: {'cnt_retweet': '0', 'cont_languages': 'zh-cn', 'page_action_type': '11', 'relevant_username': '秋名山街猫', 'userid': '5911198285', 'content': '鉴于日本东海岸约1,000个水箱中储存着大量放射性水，整个过程预计需要三十到四十年的时间。 岸田承诺，只有在不对人类健康和环境造成危害的情况下，才会向海洋释放废水。 RT:@秋名山街猫 国际原子能机构(IAEA)最终认可了日本将受损的福岛第一核电站处理后的核废水排入太平洋的计划。 国际原子能机构批准福岛废水排放计划，东亚各国密切关注', 'url': 'https://m.weibo.cn/status/4920146798053249', 'relevant_nickname': '秋名山街猫', 'relevant_userid': '7476360648', 'publish_time': '20230705141751', 'cnt_agree': '3', 'nickname': 'Countrycat', 'topic_id': '4920146798053249', 'cnt_comment': '1', 'username': 'Countrycat'}, Error: Expecting value: line 1 column 1 (char 0)


  2%|▏         | 3570/200676 [1:35:17<87:41:40,  1.60s/it] 


KeyboardInterrupt: 

In [19]:
post_all_data[112]

{}